In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/Group_A_Project
%ls

/content/drive/MyDrive/Colab Notebooks/Group_A_Project
 Amazon_Unlocked_Mobile.csv            Pre-poster.pptx
 Amazon_Unlocked_Mobile.csv_bert.pth  'Pre-poster Slide.gslides'
 BERT_Visualization.ipynb              Project.ipynb
'Copy of IMDB.ipynb'                  'Project Timeline.gsheet'
 IMDB.ipynb                            RAG.ipynb
 MacRumors.gsheet                      Trustpilot_commemt_data.csv
'MacRumors - MacRumors.csv'            Trustpilot_commemt_data.gsheet
'Milestone (Project 開發流程).gdoc'   'Web Crawler_MacRumors.ipynb'
 my_database/                         'Web crawler - trustpilot&MAC.ipynb'
 poster_template_2024.pptx


In [ ]:
%pip install langchain-community
%pip install langchain_openai
%pip install chromadb
!pip install torch torchvision torchaudio
!pip install sentence-transformers

In [ ]:
# Load File
import pandas as pd
from langchain.schema import Document
from langchain.document_loaders import TextLoader
# loader = TextLoader('Amazon_Unlocked_Mobile.csv')
# documents = loader.load()
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
documents = df['Reviews'].dropna().tolist()
chunks = [Document(page_content=line) for line in documents]

In [ ]:
# Split Data Into Chunks
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=80,  # 調整塊大小以適應文件的內容
#     chunk_overlap=40,
#     length_function=len,
#     add_start_index=True
# )
# chunks = text_splitter.split_documents(documents)


In [ ]:
import os
API_KEY = ''
os.environ['OPENAI_API_KEY'] = API_KEY

In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [ ]:
# Data Embedding

from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

CHROMA_PATH = 'my_database'

# ------------------


embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')  # upload GPU

# use SentenceTransformer save in ChromaDB
def embed_documents(documents):
    texts = [doc.page_content for doc in documents]  # extract
    embeddings = embedding_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
    return embeddings


embeddings = embed_documents(chunks)

# # Create Chroma database
# db = Chroma.from_embeddings(chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH)
# db.persist()
# print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

# db = Chroma.from_documents(chunks,OpenAIEmbeddings(),persist_directory = CHROMA_PATH)
# db.persist()
# print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please n

Batches:   0%|          | 0/12931 [00:00<?, ?it/s]

In [ ]:
# class CustomEmbeddingFunction:
#     def __call__(self, texts):
#         return embedding_model.encode(texts, convert_to_numpy=True)

# embeddings_function = CustomEmbeddingFunction()

# # Create Chroma database
# db = Chroma.from_documents(chunks, embeddings_function, persist_directory=CHROMA_PATH)
# db.persist()
# print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

# # db = Chroma.from_documents(chunks,OpenAIEmbeddings(),persist_directory = CHROMA_PATH)
# # db.persist()
# # print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [ ]:
import pandas as pd
csv_path = 'Amazon_Unlocked_Mobile.csv'
df = pd.read_csv(csv_path)

csv_content = df.to_string(index=False)
reviews_list = df['Reviews'].dropna().tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'Amazon_Unlocked_Mobile.csv'

In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:


{review}

---

Answer the question based on the above context: {question}
"""
# Answer the question based on the above context: {question}

In [ ]:
from langchain_openai import ChatOpenAI

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI()

# Store results
results = []  #

for review in reviews_list:
    query = 'Is these reviews posative or negative ? only answer positive or negative'
    # Search the DB
    search_results = db.similarity_search_with_relevance_scores(review, k=5)

    if len(search_results) == 0 or search_results[0][1] < 0.7:
        response = 'Unable to determine sentiment'
    else:
        # Create context from search results
        context = '\n\n---\n\n'.join([doc.page_content for doc, score in search_results])
        prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)


        prompt = prompt_template.format(review=context,question= query)

        response = model.predict(prompt)

    results.append({'review': review, 'response': response})
for result in results:
    print(f"Review: {result['review'][:40]}\nResponse: {result['response']}\n")



ModuleNotFoundError: No module named 'langchain'